In [1]:
import requests, json
import threading
import pandas as pd
import flask
from flask import Flask, request
import geopy
from geopy import Nominatim

ModuleNotFoundError: No module named 'requests'

In [ ]:
key = 'fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w='

# this method helps us get long and lat of certain city

def get_latlong(b):

    city = b

    geolocator = Nominatim(user_agent = 'Safra')

    loc = geolocator.geocode(city)

    return loc.latitude , loc.longitude

In [ ]:
lat , long = get_latlong( 'Riyadh')


In [ ]:
def retrieve_places(a , c):

    """
    a : condition --- >  example : coffee , art gallery , etc ...
    c : city name
    """

    lat , long = get_latlong(c)


    if a != '':
        fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&query={a}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation"

    else:
        fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation"


    url = fields_url

    headers = {
        "Accept": "application/json",
        "Authorization": key
    }

    response = requests.get(url, headers=headers)

    data = response.json()

    df = pd.json_normalize(data['results'])

    #deleting unnecessary columns
    
    df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)


    # renaming columns


    df.rename(columns = {'location.country':'country' , 'location.region':'region'}, inplace = True)



    # filling nan values

    df = df.fillna('Not Available')



    # changing datatypes
    df = df.astype({'price': str , 'rating': str})

    data = df.to_json(orient = 'records')
    return df , data

In [ ]:
df, data_json = retrieve_places('' , 'Riyadh')
df

C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)


,fsq_id,name,rating,tel,country,region,price,description
0,50a21d4fe4b0225d6322ed03,King Abdulaziz Foundation (دارة الملك عبدالعزيز),9.3,011 401 1999,SA,Riyadh Province,Not Available,Not Available
1,5c8e4d17f709c1002ca94f02,محامص ومطاحن السوادي,9.3,Not Available,SA,Riyadh Province,1.0,Not Available
2,4ee06435e5fa0be34659e3cf,AlNaqaa (حمص النقاء),9.4,011 206 4403,SA,Riyadh Province,1.0,Not Available
3,5fca47490d6d9d74b579c8fa,Masaha Misk Art Institute (قاعة مساحة معهد مسك...,9.5,Not Available,SA,Riyadh Province,Not Available,Not Available
4,566db885498e8b884eecff76,Starbucks (ستاربكس),9.4,011 511 1838,SA,Riyadh Province,2.0,The Starbucks Experience is more than a cup of...
5,5e640a2684328a000895da0a,COYARD Coffee Roasters,9.4,Not Available,SA,Riyadh Province,Not Available,Not Available
6,60703b211fee5e329ea0f64e,Oakberry Açai,9.3,Not Available,SA,Riyadh Province,Not Available,Not Available
7,6248bb4a7da6a86c9bc265a5,Ba’a Bakehouse,9.4,Not Available,SA,Riyadh Province,1.0,Not Available
8,544feecb498ef9028459c0f9,Tamimi Markets (أسواق التميمي),9.3,Not Available,SA,Riyadh Province,Not Available,Not Available
9,5dc3325ae167fe00084f61ce,Elixir Bunn Coffee Roasters (محمصة و مقهى إكسي...,9.3,9200 13926,SA,Riyadh Province,1.0,Not Available


In [ ]:


app = Flask(__name__)

@app.route('/api' , methods = ['GET'])

def index():
    userInputb = str(request.args['query'])
    df, data_json = retrieve_places('' , userInputb)

    return data_json




if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Sep/2022 19:34:22] "GET / HTTP/1.1" 404 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [21/Sep/2022 19:34:28] "GET /api?query=Riyadh HTTP/1.1" 200 -
[2022-09-21 19:34:36,324] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1517, in full

C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [21/Sep/2022 19:43:53] "GET /api?query=Riyadh HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [21/Sep/2022 19:46:11] "GET /api?query=Riyadh HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for

127.0.0.1 - - [21/Sep/2022 19:56:15] "GET /api?query=Riy HTTP/1.1" 500 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [21/Sep/2022 19:56:15] "GET /api?query=Riyad HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
[2022-09-21 19:56:17,353] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\ap

C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [21/Sep/2022 20:28:49] "GET /api?query=Riyadh HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [21/Sep/2022 20:29:07] "GET /api?query=Riyadh HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for

...

127.0.0.1 - - [21/Sep/2022 20:49:09] "GET /api?query=r HTTP/1.1" 500 -
[2022-09-21 20:49:09,301] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\Saud\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 1377, in getresponse
    response.begin()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Saud\miniconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Saud\miniconda3\lib

127.0.0.1 - - [21/Sep/2022 20:49:09] "GET /api?query=riyadh HTTP/1.1" 500 -
[2022-09-21 20:49:16,812] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\3309281160.py", line 7, in index
    df, data_json = retrieve_places('' , userInputb)
  File "C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py", line 8, in retrieve_pl

127.0.0.1 - - [21/Sep/2022 20:49:22] "GET /api?query=Ri HTTP/1.1" 500 -
[2022-09-21 20:49:22,870] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\Saud\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 1377, in getresponse
    response.begin()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Saud\miniconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Saud\miniconda3\li

127.0.0.1 - - [21/Sep/2022 20:49:23] "GET /api?query=Riyad HTTP/1.1" 500 -
[2022-09-21 20:49:30,540] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\3309281160.py", line 7, in index
    df, data_json = retrieve_places('' , userInputb)
  File "C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py", line 8, in retrieve_pla

C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [21/Sep/2022 20:52:05] "GET /api?query=p HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_18716\2794973027.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
[2022-09-21 20:52:05,469] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  F